In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import multidict
from gurobipy import quicksum
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Defining the data

model = gp.Model("FilmProductionScheduler")

# Example data
crews = ["makeup_artist", "lighting", "camera", "sound"]
tasks = ["makeup_scene1", "lighting_scene1", "filming_scene1", "sound_scene1", "makeup_scene2", "lighting_scene2", "filming_scene2", "sound_scene2", "makeup_scene3", "lighting_scene3", "filming_scene3", "sound_scene3"]
scenes = ["scene1", "scene2", "scene3"]
time_slots = range(48)  # 48 half-hour slots in a day 
# Task durations in time slots
task_duration = {"makeup_scene1": 4, "lighting_scene1": 6, "filming_scene1": 12, "sound_scene1": 4, "makeup_scene2": 4, "lighting_scene2": 6, "filming_scene2": 12, "sound_scene2": 4, "makeup_scene3": 4, "lighting_scene3": 6, "filming_scene3": 12, "sound_scene3": 4}
# Crew costs per time slot
crew_cost = {"makeup_artist": 100, "lighting": 150, "camera": 200, "sound": 100}
# crew_availability = {
#     "makeup_artist": [(0, 48)],  # Available 0-24 hours
#     "lighting": [(16, 40)],
#     "camera": [(12, 44)],
#     "sound": [(24, 48)]
# }
# crew_availability = {
#     "makeup_artist": [(16, 40)],  # Available 0-24 hours
#     "lighting": [(16, 40)],
#     "camera": [(16, 40)],
#     "sound": [(16, 40),(42,44)]
# }
crew_availability = {
    "makeup_artist": [(0, 200)],  # Available 0-24 hours
    "lighting": [(0, 200)],
    "camera": [(0, 200)],
    "sound": [(0, 200)]
}
# crew_availability = {
#     "makeup_artist": [(0, 48)],  # Available 0-24 hours
#     "lighting": [(0, 48)],
#     "camera": [(0, 48)],
#     "sound": [(0, 48)]
# }
# Task requirements
task_eligibility = {
    "makeup_artist": ["makeup_scene1", "makeup_scene2", "makeup_scene3"],
    "lighting": ["lighting_scene1", "lighting_scene2", "lighting_scene3"],
    "camera": ["filming_scene1", "filming_scene2", "filming_scene3"],
    "sound": ["sound_scene1", "sound_scene2", "sound_scene3"]
}

Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-10


In [3]:
# Create decision variables

x = {}  # x[task, crew, start_time] = 1 if task is assigned to crew starting at time
for task in tasks:
    for crew in crews:
        for t in time_slots:
            x[task, crew, t] = model.addVar(vtype=GRB.BINARY, name=f"x_{task}_{crew}_{t}")
            # if t + task_duration[task] <= len(time_slots):  # Task fits within time horizon
            #     for start, end in crew_availability[crew]:
            #         if start <= t < end and t + task_duration[task] <= end:  # Task fits within crew availability
            #             x[task, crew, t] = model.addVar(vtype=GRB.BINARY, name=f"x_{task}_{crew}_{t}")
assignment = {}

In [4]:
# Data processing

crew_available_timeslots = {}
for crew in crews:
    crew_available_timeslots[crew] = []
    for availability in crew_availability[crew]:
        start, end = availability
        crew_available_timeslots[crew].extend(list(range(start+1, end+1)))

In [5]:
# Constraints

# Each task is assigned to exactly one crew and once only (across all active time slots)
# for task in tasks:
#     for crew in crews:
#         model.addConstr(
#             quicksum(x[task, crew, t] for t in time_slots
#                     if (task, crew, t) in x) >= task_duration[task], name=f"task_duration_{task}_{crew}"
#         )
for task in tasks:
    model.addConstr(
        quicksum(x[task, crew, t] for t in time_slots for crew in crews
                if (task, crew, t) in x) == task_duration[task], name=f"task_duration_{task}_{crew}"
    )

# Each task is assigned to designated number of crew and once only (across all active time slots)

# Each crew is assigned to at most one task at a time
for crew in crews:
    for t in time_slots:
        model.addConstr(
            quicksum(x[task, crew, t] for task in tasks 
                     if (task, crew, t) in x) <= 1, name=f"crew_{crew}_time_{t}_one_task"
        )

# Each crew is assigned to its job only
for crew in crews:
    for task in tasks:
        if task not in task_eligibility[crew]:
            model.addConstr(
                quicksum(x[task, crew, t] for t in time_slots
                         if (task, crew, t) in x) == 0, name=f"crew_{crew}_task_{task}_not_eligible"
            )

# Each crew is assigned to its available time only
for crew in crews:
    for time in time_slots:
        if time not in crew_available_timeslots[crew]:
            model.addConstr(
                quicksum(x[task, crew, time] for task in tasks
                         if (task, crew, time) in x) == 0, name=f"crew_{crew}_time_{time}_not_available"
            )

start = {}
for task in tasks:
    for crew in crews:
        for t in time_slots:
            start[task, crew, t] = model.addVar(vtype=GRB.BINARY, name=f"start_{task}_{crew}_{t}")
for task in tasks:
    for crew in crews:
        for t in time_slots:
            if t + task_duration[task] - 1 <= max(time_slots):  # Ensure within the time horizon
                model.addConstr(
                    quicksum(x[task, crew, t + k] for k in range(task_duration[task])) ==
                    task_duration[task] * start[task, crew, t],
                    name=f"link_start_to_task_{task}_{crew}_{t}"
                )
for task in tasks:
    for crew in crews:
        model.addConstr(
            quicksum(start[task, crew, t] for t in time_slots) <= 1,
            name=f"one_start_time_{task}_{crew}"
        )

# Wrong definition as it only accounts the starting time slot, when it comes to the second time slots or after of an event, it bugged. 
# # Continuity constraints: if a task is assigned to a crew, it must be assigned for the entire duration
# for task in tasks: 
#     for crew in crews:
#         for t in time_slots:
#             if t + task_duration[task] - 1 <= max(time_slots):
#                 model.addConstr(
#                     quicksum(x[task, crew, t + k] for k in range(task_duration[task])
#                              if (task, crew, t) in x) >= task_duration[task] * x[task, crew, t], name=f"continuity_{task}_{crew}_{t}"
#                 )
# for task in tasks:
#     for crew in crews:
#         for t in time_slots[:-1]:  # Exclude the last time slot
#             model.addConstr(
#                 x[task, crew, t + 1] <= x[task, crew, t] + quicksum(x[task, crew, t_prime] for t_prime in time_slots if t_prime < t),
#                 name=f"no_fragment_{task}_{crew}_{t}"
#             )

            # if (task, crew, t) in x:
            #     model.addConstr(
            #         quicksum(x[task, crew, t2] for t2 in range(t, min(t + task_duration[task], len(time_slots))))
            #         == task_duration[task] * x[task, crew, t], name=f"continuity_{task}_{crew}_{t}"
            #     )

In [6]:
scene_start = {scene: model.addVar(vtype=GRB.INTEGER, name=f"scene_start_{scene}") for scene in scenes}
scene_end = {scene: model.addVar(vtype=GRB.INTEGER, name=f"scene_end_{scene}") for scene in scenes}
scene_start_end_relaxation = 0

In [7]:
total_cost = gp.quicksum(
    x[task, crew, t] * crew_cost[crew]  # Active work cost per crew
    for task in tasks
    for crew in crews
    for t in time_slots
    if (task, crew, t) in x
)

idle_cost = 0.8  # Assume idle time costs 80% of active time

# Calculate idle costs for each crew
work_start = {}     # Binary variable: 1 if crew starts work at time t
work_end = {}       # Binary variable: 1 if crew ends work at time tfor crew in crews:
work_period = {}    # Binary variable: 1 if crew is working (including idle time) in time slot t
idle = {}           # Binary variable: 1 if crew is idle in time slot t

for crew in crews:
    for t in time_slots:
        work_start[crew, t] = model.addVar(vtype=GRB.BINARY, name=f"work_start_{crew}_{t}")
        work_end[crew, t] = model.addVar(vtype=GRB.BINARY, name=f"work_end_{crew}_{t}")
        work_period[crew, t] = model.addVar(vtype=GRB.BINARY, name=f"work_period_{crew}_{t}")
        idle[crew, t] = model.addVar(vtype=GRB.BINARY, name=f"idle_{crew}_{t}")

for crew in crews:
    for t in time_slots:
        if t > min(time_slots):
            model.addConstr(
                work_start[crew, t] >= quicksum(x[task, crew, t] for task in tasks if (task, crew, t) in x) -
                                     quicksum(x[task, crew, t-1] for task in tasks if (task, crew, t-1) in x),
                name=f"work_start_logic_{crew}_{t}"
            )
            model.addConstr(
                work_period[crew, t] >= work_period[crew, t-1] + work_start[crew, t],
                name=f"work_period_start_{crew}_{t}"
            )

        if t < max(time_slots):
            model.addConstr(
                work_end[crew, t] >= quicksum(x[task, crew, t] for task in tasks if (task, crew, t) in x) -
                                   quicksum(x[task, crew, t+1] for task in tasks if (task, crew, t+1) in x),
                name=f"work_end_logic_{crew}_{t}"
            )
            model.addConstr(
                work_period[crew, t] <= work_period[crew, t+1] + work_end[crew, t],
                name=f"work_period_end_{crew}_{t}"
            )
        
        model.addConstr(
            idle[crew, t] >= work_period[crew, t] - quicksum(x[task, crew, t] for task in tasks if (task, crew, t) in x),
                                                             name=f"idle_logic_{crew}_{t}"
        )

total_cost += quicksum(
    idle[crew, t] * crew_cost[crew] * idle_cost
    for crew in crews
    for t in time_slots
)

# Set the objective function
model.setObjective(total_cost, GRB.MINIMIZE)

In [8]:
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3276 rows, 5382 columns and 36688 nonzeros
Model fingerprint: 0x43868ddf
Variable types: 0 continuous, 5382 integer (5376 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [8e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 2647 rows and 4791 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [9]:
for task in tasks:
        for crew in crews:
            for t in time_slots:
                if (task, crew, t) in x and x[task, crew, t].x > 0.5:
                    print(f"Task {task} assigned to {crew} at time {t}")
                    

AttributeError: Unable to retrieve attribute 'x'

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

def visualize_schedule_plotly(model, x, tasks, crews, time_slots, task_duration):
    """
    Visualize the film production schedule based on the optimization results using Plotly.
    
    Parameters:
    model - Gurobi model
    x - Dictionary of decision variables
    tasks - List of tasks
    crews - List of crews
    time_slots - Range of time slots
    task_duration - Dictionary mapping tasks to their durations
    """
    # Extract schedule data from the optimization solution
    schedule_data = []
    
    # Group consecutive time slots for the same task and crew
    task_assignments = {}
    
    # First, collect all assigned time slots
    for task in tasks:
        for crew in crews:
            for t in time_slots:
                if (task, crew, t) in x and x[task, crew, t].x > 0.5:
                    key = (task, crew)
                    if key not in task_assignments:
                        task_assignments[key] = []
                    task_assignments[key].append(t)
    
    # Now, group consecutive time slots
    for (task, crew), slots in task_assignments.items():
        sorted_slots = sorted(slots)
        current_start = sorted_slots[0]
        current_duration = 1
        
        for i in range(1, len(sorted_slots)):
            if sorted_slots[i] == sorted_slots[i-1] + 1:
                # Consecutive time slot
                current_duration += 1
            else:
                # Non-consecutive, save the current segment and start a new one
                scene_number = task.split('_')[-1]
                scene = f"scene{scene_number.replace('scene', '')}"
                task_type = task.split('_')[0].capitalize()
                
                schedule_data.append({
                    "Crew": crew,
                    "Task": task,
                    "TaskType": task_type,
                    "Start": current_start,
                    "Duration": current_duration,
                    "End": current_start + current_duration,
                    "Scene": scene
                })
                current_start = sorted_slots[i]
                current_duration = 1
        
        # Add the last segment
        scene_number = task.split('_')[-1]
        scene = f"scene{scene_number.replace('scene', '')}"
        task_type = task.split('_')[0].capitalize()
        
        schedule_data.append({
            "Crew": crew,
            "Task": task,
            "TaskType": task_type,
            "Start": current_start,
            "Duration": current_duration,
            "End": current_start + current_duration,
            "Scene": scene
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(schedule_data)
    
    # Define crew names for display
    crew_display_names = {
        "makeup_artist": "Makeup Artist",
        "lighting": "Lighting",
        "camera": "Camera",
        "sound": "Sound"
    }
    
    # Add display names to dataframe
    df["CrewDisplay"] = df["Crew"].apply(lambda x: crew_display_names.get(x, x))
    
    # Define colors for each scene
    scene_colors = {
        "scene1": "#4e79a7",  # blue
        "scene2": "#f28e2c",  # orange
        "scene3": "#e15759"   # red
    }
    
    # Function to convert time slots to readable time
    def format_time(time_slot):
        hours = time_slot // 2
        minutes = (time_slot % 2) * 30
        am_pm = "AM" if hours < 12 else "PM"
        display_hour = hours % 12
        if display_hour == 0:
            display_hour = 12
        return f"{display_hour}:{minutes:02d} {am_pm}"
    
    # Add formatted start and end times
    df["StartTime"] = df["Start"].apply(format_time)
    df["EndTime"] = df["End"].apply(format_time)
    
    # Create hover text
    df["HoverText"] = df.apply(
        lambda row: f"<b>{row['TaskType']} ({row['Scene'].replace('scene', 'Scene ')})</b><br>" +
                   f"Crew: {row['CrewDisplay']}<br>" +
                   f"Time: {row['StartTime']} - {row['EndTime']}<br>" +
                   f"Duration: {row['Duration'] * 30} minutes",
        axis=1
    )
    
    # Sort the crews in a specific order
    crew_order = ["makeup_artist", "lighting", "camera", "sound"]
    df["CrewOrder"] = df["Crew"].apply(lambda x: crew_order.index(x) if x in crew_order else 999)
    df = df.sort_values("CrewOrder")
    
    # Create figure
    fig = go.Figure()
    
    # Add AM/PM background areas
    fig.add_shape(
        type="rect",
        x0=0, y0=-0.5,
        x1=24, y1=len(crew_order) - 0.5,
        fillcolor="rgba(230, 230, 250, 0.3)",
        line=dict(width=0),
        layer="below"
    )
    fig.add_shape(
        type="rect",
        x0=24, y0=-0.5,
        x1=48, y1=len(crew_order) - 0.5,
        fillcolor="rgba(250, 240, 230, 0.3)",
        line=dict(width=0),
        layer="below"
    )
    
    # Add tasks for each scene
    for scene in ["scene1", "scene2", "scene3"]:
        scene_df = df[df["Scene"] == scene]
        
        fig.add_trace(go.Bar(
            x=scene_df["Duration"],
            y=scene_df["CrewDisplay"],
            orientation='h',
            base=scene_df["Start"],
            name=scene.replace("scene", "Scene "),
            marker=dict(color=scene_colors[scene]),
            hovertext=scene_df["HoverText"],
            hoverinfo="text",
            text=scene_df["TaskType"],
            textposition="inside",
            insidetextanchor="middle",
            textfont=dict(color="white", size=12),
            width=0.6
        ))
    
    # Customize layout
    fig.update_layout(
        title={
            'text': 'Film Production Schedule',
            'y': 0.98,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=20, color='black')
        },
        xaxis=dict(
            title="Time",
            tickmode="array",
            tickvals=list(range(0, 49, 4)),
            ticktext=[format_time(t) for t in range(0, 49, 4)],
            range=[0, 48],
            gridcolor='lightgray',
            gridwidth=1,
            showgrid=True
        ),
        yaxis=dict(
            title="Crew",
            categoryorder="array",
            categoryarray=[crew_display_names[crew] for crew in crew_order]
        ),
        barmode='overlay',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='rgba(0, 0, 0, 0.2)',
            borderwidth=1
        ),
        plot_bgcolor='white',
        height=600,
        width=1200,
        margin=dict(l=100, r=50, t=100, b=100)
    )
    
    # Add annotations for AM/PM periods
    fig.add_annotation(
        x=12, y=-1,
        text="Morning (12AM - 12PM)",
        showarrow=False,
        font=dict(size=14),
        xanchor="center"
    )
    
    fig.add_annotation(
        x=36, y=-1,
        text="Evening (12PM - 12AM)",
        showarrow=False,
        font=dict(size=14),
        xanchor="center"
    )
    
    # Add vertical line at 12pm
    fig.add_shape(
        type="line",
        x0=24, y0=-0.5,
        x1=24, y1=len(crew_order) - 0.5,
        line=dict(color="black", width=1.5, dash="dash")
    )
    
    # Add horizontal grid lines
    for i in range(len(crew_order)):
        if i < len(crew_order) - 1:  # Don't add line after the last crew
            fig.add_shape(
                type="line",
                x0=0, y0=i + 0.5,
                x1=48, y1=i + 0.5,
                line=dict(color="lightgray", width=1)
            )
    
    # Show figure
    fig.show()
    
    return df

# Example usage:
# schedule_df = visualize_schedule_plotly(model, x, tasks, crews, time_slots, task_duration)

In [ ]:
# Visualize the schedule with Plotly
schedule_df = visualize_schedule_plotly(model, x, tasks, crews, time_slots, task_duration)